# Trabalho da disciplina de **LUI**

## O objetivo do trabalho é extrair informações de uma fonte de sua própria escolha, sendo através de uma **API** ou por **Web Scraping**.

## Você deve fornecer o código com o output do dataframe e as saidas após as seguintes operações de Ordenação e Filtragem.

## Etapas:

1.   Escolher uma API ou site que contenham as informações desejadas
2.   Guardar todas essas informações extraídas em um DataFrame
3.   Executar a operação de ordenação em um atributo escolhido (ao seu critério, podendo ser em ordem numérica ou alfabética)
4.   Executar a operação de filtragem em um atributo escolhido (ao seu critério)


O trabalho pode conter até 5~6 pessoas por grupo, não mais do que isso!



#CEP's e Localidades no Brasil
  

---

Grupo: **Gustavo Dutra Martins**   
Objetivo:  Utilização de Web Scraping no site https://ceps.io/ e chamada a API de localidades do IBGE (https://servicodados.ibge.gov.br/api/docs/localidades) para criação de um Arquivo CSV com todos os cep's do Brasil e respectivas localidades (região, mesoregião e microregião).  

Após a criação do data frame são realizadas as atividades solicitas para o trabalho: listagem, ordenação e filtragem.  

Não foram realizadas manipulações mais elaboradas no dataframe porque entendemos que a maior dificuldade do trabalho estava na carga de cep's e junção com localidades.  

A execução do notebook é extramente demorada, cerca de 1 dia no Colab PRO. Não utilizei Threads porque precisaria mudar a implementação já testada. Existe um break comentado no método de carregamento UF's para execução pelo avaliador (caso necessário), caso seja descomentado então somente o estado do Acre será carregado (10-15 minutos).  

O link https://drive.google.com/drive/folders/1cTYMVBRRZWC407V6JRQ-2ExUdnTtCCUv?usp=sharing referencia o diretório com o resultado do processamento. O arquivo 'ceps-brasil.csv' contém todos os cep's carregados na primeira fase. O arquivo 'ceps-brasil-completo.csv' contém os ceps com as localidades (segunda fase). 

Também geramos um arquivo compactado para facilitar o download.  
  
O trabalho foi concluído com sucesso, no entanto alguns municípios não foram encontrados na base do IBGE. Não exploramos a solução de todos os problemas, mas identificamos a presença de caracteres especiais e outras situações onde os municípios não estavam listados no IBGE, acreditamos que os dados podem estar referenciando diferente data base, o que explicaria a diferença.  
  
O trabalho também foi publicado no github - https://github.com/gdutramartins/bi-master-lui

#### Preparação para execução

*   Instalação e importação de bibliotecas



In [ ]:
pip install Unidecode

In [ ]:
import requests
from requests.models import Response
import re
from tqdm import tqdm
from tqdm.auto import trange
import time
import os

from dataclasses import dataclass
from typing import List, Dict

from bs4 import BeautifulSoup
from bs4.element import Tag
import unidecode

import pandas as pd
from pandas.core.series import Series
import json

#### Constantes

In [ ]:
URL_RAIZ_CEP = "https://ceps.io"
CSV_SEP = '|'
DATASET_CEP_PATH:str = 'ceps-brasil.csv'

## 1 - Classes de Dados (DTO's)
criadas para armazenar dados carregados por Web Scraping (CEP) ou API (localidades Brasil)



In [ ]:
# Último nível de dado carregado pelo Scraping de CEP. Bairro contém uma lista de Cep`s.
@dataclass
class Cep:
    numero: int
    link: str
    localizacao: str

    def to_csv(self) -> str:
        return self.numero + CSV_SEP + self.localizacao + CSV_SEP + self.link

In [ ]:
# Bairro carregado pelo Scraping de CEP. Contém a lista de Cep's. Municipio contém uma lista de Bairros.
@dataclass
class Bairro:
    nome: str
    link: str
    ceps: List[Cep]

    def to_csv(self) -> str:
        return self.nome + CSV_SEP + self.link

In [ ]:
# Municipio carregado pelo Scraping de CEP. Contém a lista de bairros. UF contém uma lista de Municipios
@dataclass
class Municipio:
    nome: str
    link: str
    bairros: List[Bairro]

    def to_csv(self) -> str:
        return self.nome + CSV_SEP + self.link

In [ ]:
# Estado carregado pelo Scraping de CEP. Primeiro nível do carregamento
@dataclass
class UF:
    nome: str
    link: str
    municipios: List[Municipio]

    def to_csv(self) -> str:
        return self.nome + CSV_SEP + self.link

In [ ]:
# Municipio carregado pela API do IBGE
@dataclass
class MunicipioIbge:
    nome: str
    uf: str
    microregiao: str
    mesoregiao: str
    regiao: str

In [ ]:
class CepExtractor:
    """
    Contém toda a lógica para carregamento de CEP`s utilizando Web Scraping no site https://ceps.io .
    Métodos que não devem ser chamados externamente (privados) contém o prefixo '_'
    Devido ao tempo gasto no carregamento os Ceps são exportados para arquivos CSV após a finalização do processamento de cada UF.
    A junção dos ceps em um arquivo único está em célula isolada desse notebook, devendo ser executada após o carregamento de todas as UF's.

    """
    def __init__(self, url_raiz: str):
        self.url_raiz = url_raiz
        self.ufs: List[UF] = []       

    def extrair_ceps(self, somente_uf: bool=False, somente_municipio: bool=False, somente_bairro: bool=False) -> None:
        """
         Método inicial da extração, permite extrações parciais de acordo com o parâmetro utilizado, sendo default carregamento completo.

         Arguments:
            somente_uf(bool): indica se o método deve parar a execução após o carregamento das uf's
            somente_municipio(bool): indica se o método deve parar a execução após o carregamento dos municipios
            somente_bairro(bool): indica se o método deve parar a execução após o carregamento dos baixos
        
        Returns:
            None
        """
        self.ufs = self.__extrair_ufs()
        if somente_uf:
            return
        self.__extrair_municipios()
        if somente_municipio:
            return
        self.__extrair_bairros()
        if somente_bairro:
            return
        self.__extrair_ceps()

    def __extrair_ufs(self):
        """"
        Extração das UF`s.
        """
        response: Response = requests.get(self.url_raiz)
        ufs_carregadas: List[UF] = []
        soup = BeautifulSoup(response.text, 'lxml')
        lista_tag_uf: List[Tag] = soup.find_all("a", class_="box")
        
        for tag in tqdm(lista_tag_uf, desc='Carregando UFs '):
            nome_uf = unidecode.unidecode(tag.text.strip().replace("\n",""))
            link = self.url_raiz + tag['href'] + 'related'
            uf:UF = UF(nome=nome_uf, link=link, municipios=None)
            ufs_carregadas.append(uf)
            # Caso deseje executar para teste, com esse break carregará somente o Acre
            #break

        return ufs_carregadas

    def __extrair_municipios(self) -> None:
        """"
        Extração dos Municipíos.
        """

        for uf in tqdm(self.ufs, desc='Carregando Municipios', position=0):
            response = requests.get(uf.link)
            response_regex_formated = re.sub(r"<h3>Outros estados([\w\s\W])*", "", response.text)

            soup = BeautifulSoup(response_regex_formated, 'lxml')
            lista_tag_municipios: List[Tag] = soup.find_all("a", class_="box")

            uf.municipios = []
            for tag in lista_tag_municipios:
                 nome_municipio = unidecode.unidecode(tag.text.strip().replace("\n",""))
                 link = self.url_raiz + tag['href'] + 'related'
                 municipio: Municipio = Municipio(nome=nome_municipio, link= link, bairros=None)
                 uf.municipios.append(municipio)            
            

    def __extrair_bairros(self) -> None:
        """"
        Extração dos Bairros.
        """
        
        for uf in self.ufs:
            for ind_mun in trange(len(uf.municipios), desc=f'Carregando Bairros UF {uf.nome}', leave=False) :
                mun = uf.municipios[ind_mun]
                response = requests.get(mun.link)
                response_regex_formated = re.sub(r"<h3>Outras cidades([\w\s\W])*","", response.text)

                soup = BeautifulSoup(response_regex_formated, 'lxml')
                lista_tag_bairros: List[Tag] = soup.find_all("a", class_="box")

                mun.bairros = []
                for ind_tag in trange(len(lista_tag_bairros), desc=f'Carregando Bairros do Municipio {mun.nome}', leave=False):
                    tag = lista_tag_bairros[ind_tag]
                    nome_bairro = tag.text.strip().replace("\n","").replace(CSV_SEP,"")
                    link = self.url_raiz + tag['href'] + 'related'
                    bairro: Bairro = Bairro(nome=nome_bairro, link= link, ceps=None)
                    mun.bairros.append(bairro)      

    def __extrair_ceps(self) -> None:
        """"
        Extração dos CEP`s.
        """

        for uf in self.ufs:
            for ind_mun in trange(len(uf.municipios), desc=f'Carregando CEP UF - {uf.nome}', leave=False) :
                mun = uf.municipios[ind_mun]
                for ind_bairro in trange(len(mun.bairros), desc=f'Carregando CEP Municipio - {mun.nome}', leave=False):
                    bairro = mun.bairros[ind_bairro]
                    response = requests.get(bairro.link)
                    # o retorno do request de CEP traz "Outros bairros na cidade", sendo a configuração de elementos html identica
                    # Vamos aplicar um filtro para retirar essa parte do Html

                    response_regex_formated = re.sub(r"<h3>Outros bairros([\w\s\W])*", "", response.text)
                    soup = BeautifulSoup(response_regex_formated, 'lxml')
                    lista_tag_ceps: List[Tag] = soup.find_all("a", class_="box")

                    bairro.ceps =[]
                    for tag in lista_tag_ceps:
                        tag_numero = tag.find("h4")
                        tag_localizacao = tag.find("p")
                        
                        if not(tag_numero is None):
                             numero = tag_numero.text.strip().replace("\n","")
                        else:
                            numero = 'N/A'
                        if not(tag_localizacao is None):
                            localizacao = tag_localizacao.text.strip().replace("\n","").replace(CSV_SEP,"")
                        else:
                            localizacao = 'N/A'
                        
                        link = self.url_raiz + tag['href'] 
                        cep: Cep = Cep(numero=numero, link= link, localizacao=localizacao)
                        bairro.ceps.append(cep)
            #salvar arquivo de bairros após processamento, muito custoso reprocessar e Colab reinicia as vezes
            self.__exporta_uf_to_csv(uf, uf.nome + '-' + 'cep' + '.csv')

    def __exporta_uf_to_csv(self, uf:UF, path: str) -> None:
        """
        Exportação da UF para arquivo CSV

        Arguments:
            uf(UF): Objeto com os dados da UF.
            path(str): Caminho para salvar a exportação da UF.
        """
        with open(path, 'w') as csv_file:
            for mun in uf.municipios:
                for bairro in mun.bairros:
                    for cep in bairro.ceps:
                        csv_file.write(uf.to_csv() + CSV_SEP + mun.to_csv() + CSV_SEP + bairro.to_csv() + CSV_SEP + cep.to_csv() + '\n')       

In [ ]:
"""
    Execução da extração de CEP.
    Embora as barras de progressão fiquem listadas no output da célula após o salvamento, durante a execução elas são sobrepostas, aparecendo somente uma por UF e outra por Municipio.
"""
cep_extractor = CepExtractor(URL_RAIZ_CEP)
cep_extractor.extrair_ceps()

Carregando Municipios: 100%|██████████| 1/1 [00:00<00:00,  1.37it/s]


Carregando Bairros UF Acre:   0%|          | 0/23 [00:00<?, ?it/s]

Carregando Bairros do Municipio Acrelandia:   0%|          | 0/1 [00:00<?, ?it/s]

Carregando Bairros do Municipio Assis Brasil:   0%|          | 0/1 [00:00<?, ?it/s]

Carregando Bairros do Municipio Brasileia:   0%|          | 0/1 [00:00<?, ?it/s]

...

Carregando CEP Municipio - Santa Rosa do Purus:   0%|          | 0/1 [00:00<?, ?it/s]

Carregando CEP Municipio - Sena Madureira:   0%|          | 0/1 [00:00<?, ?it/s]

Carregando CEP Municipio - Senador Guiomard:   0%|          | 0/1 [00:00<?, ?it/s]

Carregando CEP Municipio - Tarauaca:   0%|          | 0/1 [00:00<?, ?it/s]

Carregando CEP Municipio - Xapuri:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
  # Concatenação dos arquivos csv parciais (por uf) em um único arquivo CSV com todos os cep's do Brasil
  with open('ceps-brasil.csv', 'w') as outfile:
    #colunas
    outfile.write('UF' + CSV_SEP +  'Link UF'+ CSV_SEP + 'Municipio'+ CSV_SEP + 'Link Municipio' + CSV_SEP + 'Bairro' + CSV_SEP + 'Link Bairro' +
                  CSV_SEP + 'CEP' + CSV_SEP + 'Localizacao' + CSV_SEP + 'Link CEP' + '\n')
    for uf in cep_extractor.ufs:
        path_uf_file = uf.nome + '-' + 'cep' + '.csv'
        with open(path_uf_file) as infile:
            for line in infile:
                outfile.write(line)



---





## 2 - Join com dados obetidos da API do IBGE

In [ ]:
# leitura do arquivo com todos os cep's do Brasil (Pandas)
df = pd.read_csv(DATASET_CEP_PATH, sep=CSV_SEP)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
# quantidade de registros
len(df)

1126073

In [ ]:
# primeiros registros
df.head()

,UF,Link UF,Municipio,Link Municipio,Bairro,Link Bairro,CEP,Localizacao,Link CEP
0,Acre,https://ceps.io/AC/related,Acrelandia,https://ceps.io/AC/Acrelandia/related,Centro,https://ceps.io/AC/Acrelandia/Centro/related,69945970,"Avenida Paraná, 296",https://ceps.io/AC/Acrelandia/Centro/Avenida-P...
1,Acre,https://ceps.io/AC/related,Assis Brasil,https://ceps.io/AC/Assis-Brasil/related,Centro,https://ceps.io/AC/Assis-Brasil/Centro/related,69935970,"Rua Dom Giocondo Maria Grotte, 230",https://ceps.io/AC/Assis-Brasil/Centro/Rua-Dom...
2,Acre,https://ceps.io/AC/related,Brasileia,https://ceps.io/AC/Brasileia/related,Centro,https://ceps.io/AC/Brasileia/Centro/related,69932970,"Avenida Prefeito Rolando Moreira, 170",https://ceps.io/AC/Brasileia/Centro/Avenida-Pr...
3,Acre,https://ceps.io/AC/related,Bujari,https://ceps.io/AC/Bujari/related,Centro,https://ceps.io/AC/Bujari/Centro/related,69926970,"Rua Expedito Pereira de Souza, 971",https://ceps.io/AC/Bujari/Centro/Rua-Expedito-...
4,Acre,https://ceps.io/AC/related,Campinas (Placido de Castro),https://ceps.io/AC/Campinas-Placido-de-Castro/...,Centro,https://ceps.io/AC/Campinas-Placido-de-Castro/...,69929970,Rua Kaxinawás,https://ceps.io/AC/Campinas-Placido-de-Castro/...


In [ ]:
# ultimos registros
df.tail()

,UF,Link UF,Municipio,Link Municipio,Bairro,Link Bairro,CEP,Localizacao,Link CEP
1126068,Tocantins,https://ceps.io/TO/related,Tupirama,https://ceps.io/TO/Tupirama/related,Setor Central,https://ceps.io/TO/Tupirama/Setor-Central/related,77704970,"Rua João Batista Magalhães, 7",https://ceps.io/TO/Tupirama/Setor-Central/Rua-...
1126069,Tocantins,https://ceps.io/TO/related,Tupiratins,https://ceps.io/TO/Tupiratins/related,Setor Central,https://ceps.io/TO/Tupiratins/Setor-Central/re...,77744980,"Avenida Principal, s/n",https://ceps.io/TO/Tupiratins/Setor-Central/Av...
1126070,Tocantins,https://ceps.io/TO/related,Tupiratins,https://ceps.io/TO/Tupiratins/related,Setor Central,https://ceps.io/TO/Tupiratins/Setor-Central/re...,77743970,"Avenida Tocantins, 681",https://ceps.io/TO/Tupiratins/Setor-Central/Av...
1126071,Tocantins,https://ceps.io/TO/related,Wanderlandia,https://ceps.io/TO/Wanderlandia/related,Setor Central,https://ceps.io/TO/Wanderlandia/Setor-Central/...,77860970,"Avenida Gomes Ferreira, 727",https://ceps.io/TO/Wanderlandia/Setor-Central/...
1126072,Tocantins,https://ceps.io/TO/related,Xambioa,https://ceps.io/TO/Xambioa/related,Setor Central,https://ceps.io/TO/Xambioa/Setor-Central/related,77880970,"Avenida Presidente Vargas, 79",https://ceps.io/TO/Xambioa/Setor-Central/Aveni...


In [ ]:
# Obtenção de dados no formato Json pela API do IBGE de localidades
response_municipios_ibge = requests.get('https://servicodados.ibge.gov.br/api/v1/localidades/municipios')
json_municipios = json.loads(response_municipios_ibge.text)

In [ ]:
# Tratamento dos municipios retirando caracteres acentuados para facilitar o join que será realizado com os municipios/UF`s carregados pelo site de CEPS
dict_uf: Dict[str,List[MunicipioIbge]] = {}

for j_municipio in json_municipios:
    nome_municipio = unidecode.unidecode(j_municipio['nome'])
    nome_uf = unidecode.unidecode(j_municipio['microrregiao']['mesorregiao']['UF']['nome'])
    microregiao =j_municipio['microrregiao']['nome']
    mesoregiao = j_municipio['microrregiao']['mesorregiao']['nome']
    regiao = j_municipio['regiao-imediata']['regiao-intermediaria']['UF']['regiao']['nome']
    
    if not nome_uf in dict_uf.keys():
        dict_uf[nome_uf] = []

    dict_uf[nome_uf].append(MunicipioIbge(nome=nome_municipio,
                                           uf=nome_uf,
                                           microregiao=microregiao, 
                                           mesoregiao=mesoregiao,
                                           regiao=regiao))
    

In [ ]:
# Função para carregamento dos dados de localidade no pandas de ceps, unindo os dados de CEP com Localidades.
# Trata a possibilidade de não encontrar a UF ou o Município
def apply_ibge_metadata(row: Series):
    lista_mun: List[MunicipioIbge] = None
    uf = row['UF']
    if uf in dict_uf.keys():
        lista_mun = dict_uf[uf]
    else:
        return 'UF N/A', 'UF N/A', 'UF N/A'
        print(f"UF nao encontrada {row['UF']}")
    for mun in lista_mun:
        if mun.nome.lower() == row['Municipio'].lower():
            return mun.microregiao, mun.mesoregiao, mun.regiao
    # nao achou trata os casos que o municipio correto vem entre ()
    groups_regex = re.findall('\(([^)]*)\)', row['Municipio'])
    if len(groups_regex) > 0:
        mun_regex = groups_regex[0]
        for mun in lista_mun:
            if mun.nome.lower() == mun_regex.lower():
                return mun.microregiao, mun.mesoregiao, mun.regiao

    print(f"Municipio nao encontrado {row['UF']}/{row['Municipio']}")
    return 'Mun N/A', 'Mun N/A', 'Mun N/A'

In [ ]:
#Aplica o merge e cria o Data Frame completo, juntando CEP com Localidade (Região, Meso Região e Micro Região)
# Existem municípios que não foram encontrados, estes são listados para tratamento em momento oportuno, não será tratado nesse trabalho
applied_df = df.apply(lambda row: apply_ibge_metadata(row), axis='columns', result_type='expand')
applied_df.rename(columns={0: 'Microregiao', 1: 'Mesoregiao', 2: 'Regiao'}, inplace=True)
df_completo = pd.concat([df, applied_df], axis='columns')

Municipio nao encontrado Alagoas/Olho DAgua das Flores
Municipio nao encontrado Alagoas/Olho DAgua do Casado
Municipio nao encontrado Alagoas/Olho DAgua Grande
Municipio nao encontrado Alagoas/Tanque DArca
Municipio nao encontrado Bahia/Dias DAvila
Municipio nao encontrado Bahia/Muquem de Sao Francisco
Municipio nao encontrado Bahia/Muquem de Sao Francisco
Municipio nao encontrado Bahia/Santa Teresinha
Municipio nao encontrado Distrito Federal/Taguatinga
Municipio nao encontrado Distrito Federal/Taguatinga
Municipio nao encontrado Distrito Federal/Taguatinga
Municipio nao encontrado Distrito Federal/Taguatinga
Municipio nao encontrado Distrito Federal/Taguatinga
Municipio nao encontrado Distrito Federal/Taguatinga
Municipio nao encontrado Distrito Federal/Taguatinga
Municipio nao encontrado Distrito Federal/Taguatinga
Municipio nao encontrado Goias/Bom Jesus
Municipio nao encontrado Goias/Forte (Sao Joao DAlianca)
Municipio nao encontrado Goias/Sao Joao DAlianca
Municipio nao encontrad

In [ ]:
# geração do arquivo CSV completo
df_completo.to_csv('ceps-brasil-completo.csv', sep=CSV_SEP)

In [ ]:
# Primeiros registros do data frame completo
df_completo.head()

,UF,Link UF,Municipio,Link Municipio,Bairro,Link Bairro,CEP,Localizacao,Link CEP,Microregiao,Mesoregiao,Regiao
0,Acre,https://ceps.io/AC/related,Acrelandia,https://ceps.io/AC/Acrelandia/related,Centro,https://ceps.io/AC/Acrelandia/Centro/related,69945970,"Avenida Paraná, 296",https://ceps.io/AC/Acrelandia/Centro/Avenida-P...,Rio Branco,Vale do Acre,Norte
1,Acre,https://ceps.io/AC/related,Assis Brasil,https://ceps.io/AC/Assis-Brasil/related,Centro,https://ceps.io/AC/Assis-Brasil/Centro/related,69935970,"Rua Dom Giocondo Maria Grotte, 230",https://ceps.io/AC/Assis-Brasil/Centro/Rua-Dom...,Brasiléia,Vale do Acre,Norte
2,Acre,https://ceps.io/AC/related,Brasileia,https://ceps.io/AC/Brasileia/related,Centro,https://ceps.io/AC/Brasileia/Centro/related,69932970,"Avenida Prefeito Rolando Moreira, 170",https://ceps.io/AC/Brasileia/Centro/Avenida-Pr...,Brasiléia,Vale do Acre,Norte
3,Acre,https://ceps.io/AC/related,Bujari,https://ceps.io/AC/Bujari/related,Centro,https://ceps.io/AC/Bujari/Centro/related,69926970,"Rua Expedito Pereira de Souza, 971",https://ceps.io/AC/Bujari/Centro/Rua-Expedito-...,Rio Branco,Vale do Acre,Norte
4,Acre,https://ceps.io/AC/related,Campinas (Placido de Castro),https://ceps.io/AC/Campinas-Placido-de-Castro/...,Centro,https://ceps.io/AC/Campinas-Placido-de-Castro/...,69929970,Rua Kaxinawás,https://ceps.io/AC/Campinas-Placido-de-Castro/...,Rio Branco,Vale do Acre,Norte


In [ ]:
# Colunas
df_completo.columns

Index(['UF', 'Link UF', 'Municipio', 'Link Municipio', 'Bairro', 'Link Bairro',
       'CEP', 'Localizacao', 'Link CEP', 'Microregiao', 'Mesoregiao',
       'Regiao'],
      dtype='object')

In [ ]:
# Ordena o data frame pelos campos de localidade
df_ordenado = df_completo.sort_values(by=['Regiao','Mesoregiao','Microregiao'])
df_ordenado.head(10)

,UF,Link UF,Municipio,Link Municipio,Bairro,Link Bairro,CEP,Localizacao,Link CEP,Microregiao,Mesoregiao,Regiao
206772,Goias,https://ceps.io/GO/related,Adelandia,https://ceps.io/GO/Adelandia/related,Jardim Progresso,https://ceps.io/GO/Adelandia/Jardim-Progresso/...,76159970,"Avenida Abel Luis Pereira, 178",https://ceps.io/GO/Adelandia/Jardim-Progresso/...,Anicuns,Centro Goiano,Centro-Oeste
209421,Goias,https://ceps.io/GO/related,Americano do Brasil,https://ceps.io/GO/Americano-do-Brasil/related,Setor Central,https://ceps.io/GO/Americano-do-Brasil/Setor-C...,76169970,"Avenida Goiás, 1333",https://ceps.io/GO/Americano-do-Brasil/Setor-C...,Anicuns,Centro Goiano,Centro-Oeste
215324,Goias,https://ceps.io/GO/related,Anicuns,https://ceps.io/GO/Anicuns/related,Capelinha,https://ceps.io/GO/Anicuns/Capelinha/related,76179970,"Rua São José, s/n",https://ceps.io/GO/Anicuns/Capelinha/Rua-Sao-J...,Anicuns,Centro Goiano,Centro-Oeste
215325,Goias,https://ceps.io/GO/related,Anicuns,https://ceps.io/GO/Anicuns/related,Setor Central,https://ceps.io/GO/Anicuns/Setor-Central/related,76170970,"Avenida Tocantins, 1200",https://ceps.io/GO/Anicuns/Setor-Central/Aveni...,Anicuns,Centro Goiano,Centro-Oeste
222373,Goias,https://ceps.io/GO/related,Aurilandia,https://ceps.io/GO/Aurilandia/related,Setor Central,https://ceps.io/GO/Aurilandia/Setor-Central/re...,76124970,"Rua Capitão João Gomes Porto, s/n Quadra 46 Lo...",https://ceps.io/GO/Aurilandia/Setor-Central/Ru...,Anicuns,Centro Goiano,Centro-Oeste
222374,Goias,https://ceps.io/GO/related,Avelinopolis,https://ceps.io/GO/Avelinopolis/related,Setor Irani,https://ceps.io/GO/Avelinopolis/Setor-Irani/re...,75395970,"Avenida Anicuns, 85",https://ceps.io/GO/Avelinopolis/Setor-Irani/Av...,Anicuns,Centro Goiano,Centro-Oeste
222391,Goias,https://ceps.io/GO/related,Buriti de Goias,https://ceps.io/GO/Buriti-de-Goias/related,Setor Central,https://ceps.io/GO/Buriti-de-Goias/Setor-Centr...,76154970,"Avenida Deputado Solon Amaral, 825",https://ceps.io/GO/Buriti-de-Goias/Setor-Centr...,Anicuns,Centro Goiano,Centro-Oeste
222392,Goias,https://ceps.io/GO/related,Buriti de Goias,https://ceps.io/GO/Buriti-de-Goias/related,Setor Central,https://ceps.io/GO/Buriti-de-Goias/Setor-Centr...,76154971,"Rua do Comércio, s/n",https://ceps.io/GO/Buriti-de-Goias/Setor-Centr...,Anicuns,Centro Goiano,Centro-Oeste
227194,Goias,https://ceps.io/GO/related,Choupana (Anicuns),https://ceps.io/GO/Choupana-Anicuns/related,Choupana,https://ceps.io/GO/Choupana-Anicuns/Choupana/r...,76172970,"Rua Goiás, s/n",https://ceps.io/GO/Choupana-Anicuns/Choupana/R...,Anicuns,Centro Goiano,Centro-Oeste
228380,Goias,https://ceps.io/GO/related,Firminopolis,https://ceps.io/GO/Firminopolis/related,Setor Central,https://ceps.io/GO/Firminopolis/Setor-Central/...,76109970,"Avenida Rui Barbosa, 684",https://ceps.io/GO/Firminopolis/Setor-Central/...,Anicuns,Centro Goiano,Centro-Oeste


In [ ]:
# Filtro do dataframe com um conjunto de CEP`s. Utilizei CEP`s que conhecia e funcionou perfeitamente.
df_filtro = df_completo[df['CEP'].isin([22631350,22740250,20031917])]
df_filtro.head()

,UF,Link UF,Municipio,Link Municipio,Bairro,Link Bairro,CEP,Localizacao,Link CEP,Microregiao,Mesoregiao,Regiao
649819,Rio de Janeiro,https://ceps.io/RJ/related,Rio de Janeiro,https://ceps.io/RJ/Rio-de-Janeiro/related,Barra da Tijuca,https://ceps.io/RJ/Rio-de-Janeiro/Barra-da-Tij...,22631350,Avenida Jornalista Ricardo Marinho,https://ceps.io/RJ/Rio-de-Janeiro/Barra-da-Tij...,Rio de Janeiro,Metropolitana do Rio de Janeiro,Sudeste
654557,Rio de Janeiro,https://ceps.io/RJ/related,Rio de Janeiro,https://ceps.io/RJ/Rio-de-Janeiro/related,Centro,https://ceps.io/RJ/Rio-de-Janeiro/Centro/related,20031917,Banco Nacional de Desenvolvimento Econômico e ...,https://ceps.io/RJ/Rio-de-Janeiro/Centro/Banco...,Rio de Janeiro,Metropolitana do Rio de Janeiro,Sudeste
665493,Rio de Janeiro,https://ceps.io/RJ/related,Rio de Janeiro,https://ceps.io/RJ/Rio-de-Janeiro/related,Pechincha,https://ceps.io/RJ/Rio-de-Janeiro/Pechincha/re...,22740250,Rua Alberto Pasqualini,https://ceps.io/RJ/Rio-de-Janeiro/Pechincha/Ru...,Rio de Janeiro,Metropolitana do Rio de Janeiro,Sudeste
